In [ ]:
import boto3
import pandas as pd
from io import stringIO #python3; python2: BytesIO

In [ ]:
AWS_ACCESS_KEY = "AKIATAMBR5AF2JM526U"
AWS_SECRET_KEY = "eTgtaB8qJ1/uZ7NvANOUpAN0DAU9Gl1U6IIooHPO"
AWS_REGION = "ap-south-1"
SCHEMA_NAME = "COVID_19"
S3_STAGING_DIR = "s3://gourav-test-bucket/output/"
S3_BUCKET_NAME = "gourav-test-bucket"
S3_OUTPUT_DIRECTORY = "output"

In [ ]:
athena_client = boto3.client(
    "athena",
    aws_access_key = AWS_ACCESS_KEY,
    aws_secret_access = AWS_SECRET_KEY,
    region_name = AWS_REGION,
)

In [ ]:
dict = {}

def download_and_load_query_result(
    client: boto3.client, query_response = dict
) -> pd.DataFrame:
    while True:
        try:
            # this function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId = query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                time.sleep(0.001)
            else:
                raise err
        temp_file_location: str = "athena_query_results.csv"
        s3_client = boto3.client (
            "s3",
            aws_access_key_id = AWS_ACCESS_KEY,
            aws_secret_access = AWS_SECRET_KEY,
            region_name = AWS_REGION
        )
        s3_client.download_file (
            S3_BUCKET_NAME,
            f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
            temp_file_location
        )
        return pd.read_csv(temp_file_location)

In [ ]:
response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM enigma_jhud",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

enigma_jhud = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM nytimes_data_in_usa_us_county",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

nytimes_data_in_usa_us_county = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM nytimes_data_in_usa_us_states",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

nytimes_data_in_usa_us_states = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM static_datacountrycode",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

static_datacountrycode = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM static_datacountypopulation",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

static_datacountypopulation = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM rearc_covid_19_testing_dataus_daily",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

rearc_covid_19_testing_dataus_daily = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM rearc_usa_hospital_beds",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

rearc_usa_hospital_beds = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM rearc_covid_19_testing_dataus_total_latest",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

rearc_covid_19_testing_dataus_total_latest = download_and_load_query_result(athena_client, response)

response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM rearc_covid_19_testing_data_states_dailystates_daily",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

rearc_covid_19_testing_data_states_dailystates_daily = download_and_load_query_result(athena_client, response)

In [ ]:
response = athena_client.start_query_execution (
    QueryString = "SELECT * FROM static_datastate_abv",
    QueryExecutionContext = {"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "ExecutionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)

static_datastate_abv = download_and_load_query_result(athena_client, response)

new_header = static_datastate_abv.iloc[0] # grab the first row for the header
static_datastate_abv = static_datastate_abv[1:] # take the data less the header row
static_datastate_abv.columns = new_header # set the header row as the new header

In [ ]:
factCovid_1 = enigma_jhud[['fips', 'province_state', 'country_region', 'confirmed', 'deaths', 'recovered', 'active']]
factCovid_2 =  rearc_covid_19_testing_data_states_dailystates_daily[['fips', 'date', 'positive', 'negative', 'hospitalized', 'hospitalizedcurrently', 'hospitalizeddischarged']]

factCovid = pd.merge(factCovid_1, factCovid_2, on='fips', how = 'inner')

In [ ]:
dimRegion1 = enigma_jhud[['fips', 'province_state', 'country_region', 'latitude', 'longitude']]
dimRegion2 = nytimes_data_in_usa_us_county[['fips', 'county', 'state']]
dimRegion = pd.merge(dimRegion1, dimRegion2, on='fips', how='inner')

In [ ]:
dimHospital = rearc_usa_hospital_beds[['fips', 'state_name', 'latitude', 'longitude', 'hq_address', 'hospital_name', 'hospital_type', 'hq_city', 'hq_state']]

In [ ]:
dimDate = rearc_covid_19_testing_data_states_dailystates_daily[['fips', 'date']]
dimDate['date'] = pd.to_datetime(dimDate['date'], format='%Y%m%d')

dimDate['year'] = dimDate['date'].dt.year
dimDate['month'] = dimDate['date'].dt.month
dimDate["day_of_week"] = dimDate['date'].dt.day_of_week

In [ ]:
bucket = 'gourav-covid-de-project' # already created on s3

In [ ]:
csv_buffer = StringIO()
factCovid.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'output/factCovid.csv').put(Body=csv_buffer.getvalue())

In [ ]:
# getting schema from a Pandas dataframe

dimDatesql = pd.io.sql.get_schema(dimDate.reset_index(), 'dimDate')
factCovidsql = pd.io.sql.get_schema(factCovid.reset_index(), 'factCovid')
dimRegionsql = pd.io.sql.get_schema(dimRegion.reset_index(), 'dimRegion')
dimHospitalsql = pd.io.sql.get_schema(dimHospital.reset_index(), 'dimHospital')

In [ ]:
#Connect to the cluster
import redshift_connector
conn = redshift_connector.connect(
    host='redshift-cluster.ctlwvzbuur6m.ap.south-1.redshift.amazonaws.com',
    database='dev',
    port=5439,
    user='awsuser',
    password='password_123'
)

conn.autocommit = True
  
# Create a Cursor object
cursor = conn.cursor()

# Query a table using the Cursor
cursor.execute("""
    CREATE TABLE "dimDate" (
        "index" INTEGER,
        "fips" INTEGER,
        "date" TIMESTAMP,
        "year" INTEGER,
        "month" INTEGER,
        "day_of_week" INTEGER
    )
""")

cursor.execute("""
    CREATE TABLE "dimHospital" (
        "index" INTEGER,
        "fips" INTEGER,
        "state_name" INTEGER,
        "latitude" REAL,
        "longitude" REAL,
        "hq_address" TEXT,
        "hospital_name" TEXT,
        "hospital_type" TEXT,
        "hq_city" TEXT,
        "hq_state" TEXT
    )
""")

cursor.execute("""
    CREATE TABLE "factCovid" (
        "index" INTEGER,
        "fips" INTEGER,
        "province_state" TEXT,
        "country_region" TEXT,
        "confirmed" REAL,
        "deaths" REAL,
        "recovered" REAL,
        "active" INTEGER,
        "date" INTEGER,
        "positive" REAL,
        "negative" REAL,
        "hospitalizedcurrently" REAL,
        "hospitalized" REAL,
        "hospitalizeddischarged" REAL
    )
""")

cursor.excute("""
    CREATE TABLE "dimRegion" (
        "index" INTEGER,
        "fips" REAL,
        "province_state" TEXT,
        "country_region" TEXT,
        "latitude" REAL,
        "longitude" REAL,
        "country" TEXT,
        "state" TEXT
    )
""")

In [ ]:
cursor.execute("""
    copy dimDate from 's3://gourav-covid-de-project/output/dimDate.csv'
    credentials 'aws_iam_role=arn:aws:iam::206986907456:role/redshift-s3-access'
    delimiter ','
    region 'ap-south-1'
    IGNOREHEADER 1
""")